## First import some necessary packages

In [52]:
import logging
import pathlib
import sys

import colorcet as cc
import dotenv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch
from bokeh.models import PanTool, WheelZoomTool

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [53]:
# # Read questions from cloud storage
questions = cd.read_questions(
    "./questions/6_cross_shore_transport.json"
    # "az://coastal-dynamics/questions/5_morphodynamics_upper_shoreface.json",
    # storage_options={"account_name": "coclico"},
)

# (Cross-shore) sediment transport
Welcome to the notebook of week 6! The main topic of this ntoebook is cross-shore sediment transport (chapter 7 of the book, with some extra attention for section 7.5 specifically). We will start with a small introduction, followed by a more detailed look into figure 7.21 from the book.

## Introduction
Remember from chapter 5 that the velocity $u$ close to the bed can be assumed to consists of a wave group-averaged component $\bar{u}$, a short-wave-averaged oscillatory component $u_{lo}$ and a short-wave component $u_{hi}$:
$$
u = \bar{u} + u_{lo} + u_{hi}
$$
We are interested in net sediment transport. We can use the third odd velocity moment as a proxy for this (which is a gross simplification, but a workable one):
$$
\left\langle u \left|u\right|^2\right\rangle = 3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle + \left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle + 3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle + ...
$$
Each of these terms is fully explained in the book (so you should definitly read up on this), but in short, each of these terms refers to the following process:
* $3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle$ : transport related to the mean current
* $\left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle$ : transport related to high-frequency waves
* $3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle$ : transport related to low-frequency waves
Once all contributions are known, an equillibrium profile might be drived mathematically. Bowen (1980) does this analyticall by balancing onshore and offshore transport terms. Conversely, each of the velocity moments can be directly measured in a flume, which is what Roelvink and Stive (1989) did.

For the present exercise, we are interested in figure 7.21 and how it changes for different wave conditions and bathymetry. However, we are limited by the lack of direct observations. Tinker et al (2009) presents a solution. They performed a large amount of measurements and fitted a shape function for both mean and oscillatory flow in the surf/shoaling zone, as well as onshore and offshore transport in the swash/surf zone. Though the paper is very interesting (and we definitly recommend checking it out), you are not required to know it for this exercise. You will hear more about it in the Coastal module as well, should you choose it!

## Define shape functions
Below the shape functions by Tinker et al (2009) are defined. You are not expected to know or remember these, they are just here to help us with the exercise.

In [27]:
def Tinker_mean(h, h_b):
    return (-120 * h_b**2) * (h / h_b)**4.3 * np.exp(-9.4 * h / h_b**0.75)

def Tinker_osci(h, h_b):
    return (2.75 * h_b**0.6) * (h / h_b)**3.5 * np.exp(-4.2 * h / h_b**1.05)

def Tinker_onsh(h, h_b):
    return 3.5 * h_b**1.9 * (h / h_b)**1.1 * np.exp(-31 * (h / h_b)**1.1)**(h_b**1.1)

def Tinker_offs(h, h_b):
    
    if h_b > 2.15:
        a_off = -3 * h_b + 4
    elif h_b <= 0.75:
        a_off = 0
    else:
        a_off = -1.25 * (h_b - 0.75)**2
        
    return a_off * h_b**1.1 * (h / h_b)**1.1 * np.exp(-5.7 * (h/h_b)**1.1)**(h_b**1.1)

def Tinker_total(h, h_b):
    return Tinker_mean(h, h_b) + Tinker_osci(h, h_b) + Tinker_onsh(h, h_b) + Tinker_offs(h, h_b)

Let's try these functions for a for different types of bathymetry, assuming some breakpoint depth $h_b$! 

### Define bathymetry functions
The functions below define the bathymetry. You can select the bathymetry you want to look at using the selector menu in the plot that is to be generated.

In [34]:
def linear(x, max_depth, margin=0.1):
    i_start = round(margin*len(x))
    i_end = round((1-margin)*len(x))

    linear_bath = np.zeros(x.shape)
    linear_bath[i_start:i_end] = np.linspace(0, max_depth, len(x[i_start:i_end]))
    linear_bath[i_end:] = max_depth

    return linear_bath


def double_linear(x, max_depth, margin=0.1):
    i_start = round(margin*len(x))
    i_low   = round((0.4) * len(x))
    i_high  = round((0.6) * len(x))
    i_end   = round((1-margin)*len(x))

    dl_bath = np.zeros(x.shape)
    dl_bath[i_start:i_low] = np.linspace(0, max_depth/2, len(x[i_start:i_low]))
    dl_bath[i_low:i_high]  = max_depth/2
    dl_bath[i_high:i_end]  = np.linspace(max_depth/2, max_depth, len(x[i_high:i_end]))
    dl_bath[i_end:]        = max_depth

    return dl_bath


def sigmoid(x, max_depth, margin=0.1):

    def smooth_step(length):
        dummy = np.linspace(0, 1, length)
        return 3*dummy**2 - 2*dummy**3
    
    i_start = round(margin*len(x))
    i_end = round((1-margin)*len(x))

    sig_bath = np.zeros(x.shape)
    sig_bath[i_start:i_end] = smooth_step(len(x[i_start:i_end])) * max_depth
    sig_bath[i_end:] = max_depth

    return sig_bath


def bar(x, max_depth, margin=0.1, bar_height=1):
    i_start_beach = round(margin*len(x))
    i_end_beach   = round((0.5 - margin)*len(x))
    
    i_start_bar1 = round((0.5 - margin * 0.5) * len(x))
    i_start_bar2 = round((0.5 + margin * 0.5) * len(x))
    i_end_bar1   = round((0.6 - margin * 0.5) * len(x))
    i_end_bar2   = round((0.6 + margin * 0.5) * len(x))

    bar_depth = max_depth - bar_height

    bar_bath = np.zeros(x.shape)
    bar_bath[i_start_beach:i_end_beach] = np.linspace(0, max_depth, len(x[i_start_beach:i_end_beach]))
    bar_bath[i_end_beach:i_start_bar1]  = max_depth
    bar_bath[i_start_bar1:i_start_bar2] = np.linspace(max_depth, bar_depth, len(x[i_start_bar1:i_start_bar2]))
    bar_bath[i_start_bar2:i_end_bar1]   = bar_depth
    bar_bath[i_end_bar1:i_end_bar2]     = np.linspace(bar_depth, max_depth, len(x[i_end_bar1:i_end_bar2]))
    bar_bath[i_end_bar2:]               = max_depth

    return bar_bath


def double_bar(x, max_depth, margin=0.1, bar_height1=1, bar_height2=1):
    i_start_beach = round(margin*len(x))
    i_end_beach   = round((0.4)*len(x))
    
    i_start_bar11 = round((0.5 - margin * 0.5) * len(x))
    i_start_bar12 = round((0.5 + margin * 0.5) * len(x))
    i_end_bar11   = round((0.6 - margin * 0.5) * len(x))
    i_end_bar12   = round((0.6 + margin * 0.5) * len(x))

    i_start_bar21 = round((0.75 - margin * 0.5) * len(x))
    i_start_bar22 = round((0.75 + margin * 0.5) * len(x))
    i_end_bar21   = round((0.85 - margin * 0.5) * len(x))
    i_end_bar22   = round((0.85 + margin * 0.5) * len(x))

    bar_depth1 = max_depth - bar_height1
    bar_depth2 = max_depth - bar_height2

    bar_bath = np.zeros(x.shape)
    bar_bath[i_start_beach:i_end_beach]   = np.linspace(0, max_depth, len(x[i_start_beach:i_end_beach]))
    bar_bath[i_end_beach:i_start_bar11]   = max_depth
    bar_bath[i_start_bar11:i_start_bar12] = np.linspace(max_depth, bar_depth1, len(x[i_start_bar11:i_start_bar12]))
    bar_bath[i_start_bar12:i_end_bar11]   = bar_depth1
    bar_bath[i_end_bar11:i_end_bar12]     = np.linspace(bar_depth1, max_depth, len(x[i_end_bar11:i_end_bar12]))
    bar_bath[i_end_bar12:i_start_bar21]   = max_depth
    bar_bath[i_start_bar21:i_start_bar22] = np.linspace(max_depth, bar_depth2, len(x[i_start_bar21:i_start_bar22]))
    bar_bath[i_start_bar22:i_end_bar21]   = bar_depth2
    bar_bath[i_end_bar21:i_end_bar22]     = np.linspace(bar_depth2, max_depth, len(x[i_end_bar21:i_end_bar22]))
    bar_bath[i_end_bar22:]                = max_depth

    return bar_bath

In [35]:
def generate_bathymetry(type, x, max_depth, margin=0.05):
    if type == 'Linear':
        bath = linear(x, max_depth, margin=margin)
    elif type == 'Double linear':
        bath = double_linear(x, max_depth, margin=margin)    
    elif type == 'Sigmoid':
        bath = sigmoid(x, max_depth, margin=margin)
    elif type == 'Bar':
        bath = bar(x, max_depth, margin=margin)
    elif type == 'Double bar':
        bath = double_bar(x, max_depth, margin=margin)
    return bath

### Define plot function

**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made! !

In [47]:
def show_transport(x, plot_where="pop-out", n_values=100, hb_start=0.8, max_depth=3, margin=0.05):
    """
    change value of 'plot_where' to:
    'inline' if you would like the plot to show in the notebook
    'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)
    """
    
    # hb_slider = pn.widgets.FloatSlider(name="Breaking depth", start=h[1], end=h[-2], value=h[int(0.8*len(h))], step=h[2] - h[1])
    hb_slider = pn.widgets.FloatSlider(name="Breaking depth", start=0.1 * max_depth, end=0.9 * max_depth, step=0.05, value=hb_start)

    bath_dropdown =  pn.widgets.Select(name="Bathymetry select", options=["Linear", "Double linear", "Sigmoid", "Bar", "Double bar"], value="Linear")

    @pn.depends(hb_slider.param.value, bath_dropdown.param.value)
    def plot(h_b, bath_type):
        h = generate_bathymetry(bath_type, x, max_depth, margin=margin)

        x_b = x[np.argmin(np.abs(h - h_b))]
        
        # print(x[np.argwhere(h==h_b)])
        # x_b = x[np.argwhere(h==h_b)][0,0]
        
        bath_plot = hv.Curve((x, -h), label='Bathymetry').opts(xlabel='x [m]', ylabel='z [m]') * \
                    hv.HLine(0, label='water level').opts(line_dash='dashed') * \
                    hv.VLine(x_b, label='location of breaking').opts(line_dash='dashed')

        bath_plot = bath_plot.opts(show_legend=True)
        
        mean_transport = Tinker_mean(h, h_b)
        osci_transport = Tinker_osci(h, h_b)
        total_transport = Tinker_total(h, h_b)
        
        transport_curves = hv.Curve((x, mean_transport), label='mean transport') * \
                           hv.Curve((x, osci_transport), label='oscillatory transport') * \
                           hv.Curve((x, total_transport), label='total transport')
        
        transport_plot = transport_curves.opts(xlabel='', ylabel='')
        
        p = (bath_plot.opts(
            height=250, width=800, show_grid=True) + \
         transport_plot.opts(
             height=250, width=1200, show_grid=True)).opts(shared_axes=False).cols(1)

        return p

    app = pn.Column(bath_dropdown, hb_slider, plot)
    
    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")

### Define cross-shore domain
Finally, we define our cross-shore domain.

In [48]:
# Set x-domain
x = np.linspace(0, 50, 1000)

### Now plot the results

Execute the cell below to generate the plot by using the function we defined above. Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by the adjusting the logging warning level. 

In [49]:
logging.getLogger().setLevel(logging.ERROR)

show_transport(x, plot_where="inline", max_depth=max_depth)


# LOOK AT DIFFERENT UNIQUE VALUES IN H ARRAY, AND USE THOSE AS OPTIONS FOR H_B SLIDER!!!

C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Column
    [0] Select(name='Bathymetry select', options=['Linear', 'Double linear'...], value='Linear')
    [1] FloatSlider(end=2.7, name='Breaking depth', start=0.30000000000000004, step=0.05, value=0.8)
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)

TODOS:
* FIX AXIS TOGETHER
* MAKE FIGURE NON-INTERACTABLE
* MAKE FIGURES MOVE TOGETHER
* PLOT OTHER TRANSPORT COMPONENTS
* ADD MARKDOWNS
* MAKE QUESTIONS

# Questions about:
- unit of Q
- interpretation of different scenarios